In [57]:
import numpy as np
import functions
import json

In [70]:
filepath = 'users.json'
username = 'Andrew'

cards_filepath = 'data/card_data.json'
card = 'Capital One Savor'

with open(filepath, 'r') as f:
    users = json.load(f)

with open(cards_filepath, 'r') as f:
    cards = json.load(f)

andrew = users[username]
andrew_spend = andrew['spending_categories']
andrew


{'credit_score': '740',
 'credit_age': '2',
 'spending_categories': {'monthly_dining': 600,
  'monthly_groceries': 100,
  'monthly_gas': 0,
  'monthly_streaming': 7,
  'monthly_online_shopping': 12,
  'monthly_drugstore/pharmacy': 5,
  'monthly_other': 200,
  'monthly_travel': 10,
  'monthly_hotel': 5,
  'monthly_hotel_portal': 6,
  'monthly_flights': 7,
  'monthly_flights_portal': 8,
  'monthly_car_rental': 9,
  'monthly_car_rental_portal': 10}}

In [67]:
savor= cards[card]
savor

{'issuer': 'Capital One',
 'annual_fee': 0,
 'earning_style': 'percent',
 'purpose': 'Cashback',
 'good_airlines': [],
 'spending_categories': {'Capital One Travel - Hotels': 5,
  'Capital One Travel - Car Rentals': 5,
  'Dining': 3,
  'Grocery': 3,
  'Other': 1}}

In [65]:
functions.calc_point_return(andrew, savor)

(2443,
 {'monthly_dining': {'amount': 600,
   'formal': 'Dining',
   'earn_rate': 3,
   'points': 1800},
  'monthly_groceries': {'amount': 100,
   'formal': 'Grocery',
   'earn_rate': 3,
   'points': 300},
  'monthly_gas': {'amount': 0, 'formal': 'Gas', 'earn_rate': 1, 'points': 0},
  'monthly_streaming': {'amount': 7,
   'formal': 'Streaming',
   'earn_rate': 1,
   'points': 7},
  'monthly_online_shopping': {'amount': 12,
   'formal': 'Online_Shopping',
   'earn_rate': 1,
   'points': 12},
  'monthly_drugstore/pharmacy': {'amount': 5,
   'formal': 'Drugstore',
   'earn_rate': 1,
   'points': 5},
  'monthly_other': {'amount': 200,
   'formal': 'Other',
   'earn_rate': 1,
   'points': 200},
  'monthly_travel': {'amount': 10,
   'formal': 'Travel',
   'earn_rate': 1,
   'points': 10},
  'monthly_hotel': {'amount': 5,
   'formal': 'Hotel',
   'earn_rate': 1,
   'points': 5},
  'monthly_hotel_portal': {'amount': 6,
   'formal': 'Capital One Travel - Hotels',
   'earn_rate': 5,
   'points':

In [9]:
# benefit_utilization_rate

# approval odds metric

In [77]:
functions.best_points_card(andrew, cards)

{'American Express Gold Card': 3125,
 'Chase Freedom Unlimited': 2460.0,
 'Capital One Savor': 2443,
 'Chase Sapphire Preferred': 2299}